In [1]:
import pretty_midi
import numpy as np
import matplotlib.pyplot as plt
import os
import math
import numpy as np
np.set_printoptions(threshold=np.inf)

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [33]:
import json

# https://stackoverflow.com/questions/26646362/numpy-array-is-not-json-serializable
with open(r'C:\Users\Arnold\Desktop\4560\CNN\dataset\mid-small_dataset\arousal.json','r') as load_f:
    y_arousal_data = json.load(load_f)
    y_arousal_data = np.asarray(y_arousal_data["arousal"])
    
with open(r'C:\Users\Arnold\Desktop\4560\CNN\dataset\mid-small_dataset\volence.json','r') as load_f:
    y_volence_data = json.load(load_f)
    y_volence_data = np.asarray(y_volence_data["volence"])
    
with open(r'C:\Users\Arnold\Desktop\4560\CNN\dataset\mid-small_dataset\midi.json','r') as load_f:
    x_data = json.load(load_f)
    x_data = np.asarray(x_data["midi"])

In [34]:
x_data = x_data.reshape(30000,50,128,1)
y_arousal_data=y_arousal_data.reshape(30000,1)
y_volence_data=y_volence_data.reshape(30000,1)

x_train_data = x_data[:3000,]
y_train_arousal_data=y_arousal_data[:3000,]
y_train_volence_data=y_volence_data[:3000,]


print(x_data.shape,y_arousal_data.shape,y_volence_data.shape)


(30000, 50, 128, 1) (30000, 1) (30000, 1)


In [35]:
input_data = layers.Input((50, 128,1))

x_2 =layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding="same")(input_data)
x_2 =layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding="same")(x_2)
x_2 =layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2),padding='valid')(x_2)

In [36]:
arousal_2 = layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding="same")(x_2)
arousal_2 = layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding="same")(arousal_2)
arousal_2 = layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2),padding='valid')(arousal_2)

arousal_2 = layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding="same")(arousal_2)
arousal_2 = layers.MaxPooling2D(pool_size=(3, 3),strides=(3, 3),padding='valid')(arousal_2)
arousal_2 = layers.Dropout(0.25)(arousal_2)

arousal_2 = layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding="same")(arousal_2)
arousal_2 = layers.MaxPooling2D(pool_size=(3, 3),strides=(3, 3),padding='valid')(arousal_2)
arousal_2 = layers.Dropout(0.25)(arousal_2)

arousal_2 = layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding="same")(arousal_2)
arousal_2 = layers.MaxPooling2D(pool_size=(1, 3),strides=(3, 3),padding='valid')(arousal_2)
arousal_2 = layers.Dropout(0.25)(arousal_2)

arousal_2 = layers.Flatten()(arousal_2)

arousal_2 = layers.Dense(256)(arousal_2)
arousal_2 = layers.Dropout(0.5)(arousal_2)

arousal_2 = layers.Dense(256)(arousal_2)
arousal_2 = layers.Dropout(0.5)(arousal_2)

arousal_2 = layers.Dense(1)(arousal_2)

In [37]:
volence_2 = layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding="same")(x_2)
volence_2 = layers.Conv2D(filters=64, kernel_size=3, activation='relu', padding="same")(volence_2)
volence_2 = layers.MaxPooling2D(pool_size=(2, 2),strides=(2, 2),padding='valid')(volence_2)

volence_2 = layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding="same")(volence_2)
volence_2 = layers.MaxPooling2D(pool_size=(3, 3),strides=(3, 3),padding='valid')(volence_2)
volence_2 = layers.Dropout(0.25)(volence_2)

volence_2 = layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding="same")(volence_2)
volence_2 = layers.MaxPooling2D(pool_size=(3, 3),strides=(3, 3),padding='valid')(volence_2)
volence_2 = layers.Dropout(0.25)(volence_2)

volence_2 = layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding="same")(volence_2)
volence_2 = layers.MaxPooling2D(pool_size=(1, 3),strides=(3, 3),padding='valid')(volence_2)
volence_2 = layers.Dropout(0.25)(volence_2)

volence_2 = layers.Flatten()(volence_2)

volence_2 = layers.Dense(256)(volence_2)
volence_2 = layers.Dropout(0.5)(volence_2)

volence_2 = layers.Dense(256)(volence_2)
volence_2 = layers.Dropout(0.5)(volence_2)

volence_2 = layers.Dense(1)(volence_2)

In [38]:
model_2 = keras.Model(inputs=input_data, outputs=[arousal_2, volence_2])

model_2.compile(loss = ["mean_squared_error","mean_squared_error"], optimizer='adam')

In [41]:
history = model_2.fit(x_train_data,[y_train_arousal_data,y_train_volence_data], validation_split=0.33, epochs=15, batch_size=10)

Train on 2009 samples, validate on 991 samples
Epoch 1/15
2009/2009 [==============================] - 142s 71ms/sample - loss: 0.0307 - dense_8_loss: 0.0143 - dense_11_loss: 0.0164 - val_loss: 0.0878 - val_dense_8_loss: 0.0478 - val_dense_11_loss: 0.0421
Epoch 2/15
2009/2009 [==============================] - 140s 70ms/sample - loss: 0.0294 - dense_8_loss: 0.0144 - dense_11_loss: 0.0151 - val_loss: 0.0956 - val_dense_8_loss: 0.0419 - val_dense_11_loss: 0.0563
Epoch 3/15
2009/2009 [==============================] - 141s 70ms/sample - loss: 0.0289 - dense_8_loss: 0.0142 - dense_11_loss: 0.0146 - val_loss: 0.0841 - val_dense_8_loss: 0.0450 - val_dense_11_loss: 0.0410
Epoch 4/15
2009/2009 [==============================] - 142s 71ms/sample - loss: 0.0262 - dense_8_loss: 0.0120 - dense_11_loss: 0.0143 - val_loss: 0.0897 - val_dense_8_loss: 0.0443 - val_dense_11_loss: 0.0470
Epoch 5/15
2009/2009 [==============================] - 141s 70ms/sample - loss: 0.0255 - dense_8_loss: 0.0125 - dens

In [ ]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [1]:
# input_shape = (16, 50, 128, 1)
# model = tf.keras.Sequential()
# model.add(layers.Conv2D(filters=8, kernel_size=3, activation='relu', padding="same", input_shape=input_shape[1:]))
# model.add(layers.BatchNormalization())
# model.add(layers.Reshape((model.output_shape[1],model.output_shape[2]*model.output_shape[3])))
# model.add(layers.Dense(8, activation='linear'))
# model.add(layers.Bidirectional(layers.GRU(8,return_sequences = True,activation = 'tanh')))
# model.add(layers.Dense(1, activation='linear'))
# model.add(layers.Flatten())
# model.add(layers.Dense(1))

# model.output_shape

input_data = layers.Input((50, 128,1))
x_1 = layers.Conv2D(filters=8, kernel_size=3, activation='relu', padding="same")(input_data)
x_1 = layers.BatchNormalization()(x_1)
x_1 = layers.Reshape((50,1024))(x_1)

arousal_1 = layers.Dense(8, activation='linear')(x_1)
arousal_1 = layers.Bidirectional(layers.GRU(8,return_sequences = True,activation = 'tanh'))(arousal_1)
arousal_1 = layers.Dense(1, activation='linear')(arousal_1)
arousal_1 = layers.Flatten()(arousal_1)
arousal_1 = layers.Dense(1)(arousal_1)

volence_1 = layers.Dense(8, activation='linear')(x_1)
volence_1 = layers.Bidirectional(layers.GRU(8,return_sequences = True,activation = 'tanh'))(volence_1)
volence_1 = layers.Dense(1, activation='linear')(volence_1)


model_1 = keras.Model(inputs=input_data, outputs=[arousal_1, volence_1])
model_1.compile(loss = rmse, optimizer='adam')

NameError: name 'layers' is not defined